# Predict Stock Prices using Recurrent Neural Network LSTM (Long Short Term Memory) model with keras

Galen Wilkerson

Directly from Tutorial:
https://www.youtube.com/watch?v=lWkFhVq9-nc


In [76]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import TimeSeriesSplit

#tscv = TimeSeriesSplit(n_splits=3) # we will need to split into training and test data 

In [77]:
dataset_train = pd.read_csv("./amdgoogle/GOOGL.csv", infer_datetime_format=True, index_col=0)

In [78]:
dataset_train.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-05-22,198.528534,199.524521,196.196198,196.946945,196.946945,3433700
2009-05-26,196.171173,202.702698,195.195190,202.382385,202.382385,6202700
2009-05-27,203.023026,206.136139,202.607605,202.982986,202.982986,6062500
2009-05-28,204.544540,206.016022,202.507507,205.405411,205.405411,5332200
2009-05-29,206.261261,208.823822,205.555557,208.823822,208.823822,5291100


In [79]:
dataset_train.shape

(2335, 6)

In [80]:
training_set = dataset_train.iloc[:, 0:2].values

In [81]:
training_set.shape

(2335, 2)

In [82]:
training_set

array([[  198.528534,   199.524521],
       [  196.171173,   202.702698],
       [  203.023026,   206.136139],
       ..., 
       [ 1244.140015,  1257.869995],
       [ 1255.900024,  1256.560059],
       [ 1255.      ,  1267.170044]])

In [83]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [87]:
X_train = []
y_train = []

for i in range(60, 1258):
    X_train.append(training_set_scaled[i - 60:i, 0])
    y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [88]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [89]:
X_train.shape

(1198, 60, 1)

In [90]:
y_train.shape

(1198,)

In [91]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [92]:
regressor = Sequential()

regressor.add(LSTM(units=50, return_sequences = True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

# add a dense layer to output one value (the next value to predict)
regressor.add(Dense(units = 1))

In [93]:
regressor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 60, 50)            10400     
_________________________________________________________________
dropout_24 (Dropout)         (None, 60, 50)            0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 60, 50)            20200     
_________________________________________________________________
dropout_25 (Dropout)         (None, 60, 50)            0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 60, 50)            20200     
_________________________________________________________________
dropout_26 (Dropout)         (None, 60, 50)            0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 50)                20200     
__________

In [94]:
# actually build the model (network)
regressor.compile(optimizer='adam', loss = 'mean_squared_error')

In [98]:
# train the network

epochs = 10  #Normally might set to 100

regressor.fit(X_train, y_train, epochs=epochs, batch_size=32)

Epoch 1/10
1198/1198 [==============================] - 12s - loss: 3.6086e-04     

## Now, model is trained, can test on other data

### Note, normally, should also parameterize model based on validation results before testing